## 类继承关系
```mermaid
classDiagram
    %% 元类层次结构
    class Data
    class SyntheticData
    class GBMData
    class YFData
    class BinanceData
    class CCXTData
    class AlpacaData
    
    %% 继承关系
    Data <|-- SyntheticData
    SyntheticData <|-- GBMData
    Data <|-- YFData
    Data <|-- BinanceData
    Data <|-- CCXTData
    Data <|-- AlpacaData
```

## class SyntheticData(Data)
生成模拟金融数据的抽象基类，其数据由数学模型生成而非从现实世界获得。

```python
class SyntheticData(Data):

    @classmethod
    def generate_symbol(cls, symbol: tp.Label, index: tp.Index, **kwargs) -> tp.SeriesFrame:
        """
        生成单个符号数据的抽象方法。
        
        Args:
            symbol (tp.Label): 符号标识符，用于标识和命名生成的数据系列，例如：'BTC_SYNTHETIC' 等
            index (tp.Index): 时间索引，定义数据生成的时间点序列
            **kwargs: 模型特定的生成参数，根据具体模型需要传递不同参数 
        Returns:
            tp.SeriesFrame: 返回pandas Series或DataFrame，包含生成的合成数据

        raise NotImplementedError

    @classmethod
    def download_symbol(cls,
                        symbol: tp.Label,
                        start: tp.DatetimeLike = 0,
                        end: tp.DatetimeLike = 'now',
                        freq: tp.Union[None, str, pd.DateOffset] = None,
                        date_range_kwargs: tp.KwargsLike = None,
                        **kwargs) -> tp.SeriesFrame:
        """ 
        对Data基类download_symbol方法的实现，生成时间索引并调用generate_symbol生成合成数据。
        Args:
            symbol (tp.Label): 符号标识
            start (tp.DatetimeLike, optional): 数据开始时间
            end (tp.DatetimeLike, optional): 数据结束时间
            freq (str/pd.DateOffset, optional): 时间频率
            date_range_kwargs (dict, optional): 传递给pd.date_range的额外参数
            **kwargs: 传递给generate_symbol方法的模型参数

        if date_range_kwargs is None:
            date_range_kwargs = {}
        # 生成时间索引：使用pd.date_range创建时间序列
        index = pd.date_range(
            start=to_tzaware_datetime(start, tz=get_utc_tz()),
            end=to_tzaware_datetime(end, tz=get_utc_tz()),
            freq=freq,
            **date_range_kwargs
        )
        if len(index) == 0:
            raise ValueError("Date range is empty")
        return cls.generate_symbol(symbol, index, **kwargs)

    def update_symbol(self, symbol: tp.Label, **kwargs) -> tp.SeriesFrame:
        """
        更新符号数据方法，在现有合成数据基础上生成新的数据。
        默认从现有数据的最后时间点开始生成新数据。

        download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
        # 设置更新的起始时间为现有数据的最后时间点
        download_kwargs['start'] = self.data[symbol].index[-1]
        kwargs = merge_dicts(download_kwargs, kwargs)
        return self.download_symbol(symbol, **kwargs)
```

## def generate_gbm_paths
几何布朗运动(**Geometric Brownian Motion**, GBM)：$$d{S_t} = \mu {S_t}dt + \sigma {S_t}d{W_t}$$
- $S_t$：$t$ 时刻的资产价格
- $μ$：漂移率，表示预期收益率
- $σ$：波动率，表示价格波动的幅度
- $dW_t$：维纳过程的微分，即布朗运动
- 性质包括
  - $E\left[ {{S_t}} \right] = {S_0}{e^{\mu t}}$
  - $Var\left[ {{S_t}} \right] = S_0^2{e^{2\mu t}}\left( {{e^{{\sigma ^2}t}} - 1} \right)$
  - $\ln \left( {\frac{{{S_t}}}{{{S_0}}}} \right) \sim N\left( {\left( {\mu  - \frac{{{\sigma ^2}}}{2}} \right)t,{\sigma ^2}t} \right)$

离散化公式为：$${S_{t + dt}} = {S_t}{e^{\left( {\mu  - \frac{1}{2}{\sigma ^2}} \right)dt + \sigma \sqrt {dt} Z}}$$
- $Z$ 是标准正态分布

参数
- `S0`: 初始价格（t=0 时刻的资产价格）。例如：100.0 表示初始股价为 100元
- `mu`: 年化收益率。例如：0.05 表示年化预期收益率为 5%
- `sigma`: 年化波动率（价格波动的标准差）。例如：0.2 表示年化波动率为 20%
- `T`: 总时间长度（单位与 mu、sigma 保持一致）。例如：如果 mu、sigma 是年化的，T=1 表示 1 年
- `M`: 时间步数（将总时间 T 分割成 M 个小段）
  - 时间步长 `dt` = T/M
  - M 越大，模拟越精确，但计算量越大
- `I`: 路径数量（需要生成的独立路径数）。用于蒙特卡洛模拟时的路径数量。
- `seed`: 随机种子，用于结果的可重现性。设置后每次运行会产生相同的随机序列

返回：形状为 `(M+1, I)` 的二维 numpy 数组
- 行数：M+1 (包含初始时刻，共 M+1 个时间点)
- 列数：I (每列代表一条独立的价格路径)
- paths[0, :] = S0 (所有路径的初始价格都是 S0)
- paths[t, i] 表示第 i 条路径在第 t 个时间点的价格

```python
def generate_gbm_paths(S0: float, mu: float, sigma: float, T: int, M: int, I: int,
                       seed: tp.Optional[int] = None) -> tp.Array2d:
    if seed is not None:
        np.random.seed(seed)

    dt = float(T) / M
    paths = np.zeros((M + 1, I), np.float64)
    paths[0] = S0
    for t in range(1, M + 1):
        rand = np.random.standard_normal(I)
        paths[t] = paths[t - 1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * rand)
    return paths
```

## class GBMData(SyntheticData)
基于几何布朗运动（GBM）模型生成金融时间序列数据。

```python
class GBMData(SyntheticData):
    @classmethod
    def generate_symbol(cls,
                        symbol: tp.Label,
                        index: tp.Index,
                        S0: float = 100.,
                        mu: float = 0.,
                        sigma: float = 0.05,
                        T: tp.Optional[int] = None,
                        I: int = 1,
                        seed: tp.Optional[int] = None) -> tp.SeriesFrame:
        """使用 GBM 模型生成指定符号的价格序列

        if T is None:
            T = len(index)
        out = generate_gbm_paths(S0, mu, sigma, T, len(index), I, seed=seed)[1:]
        if out.shape[1] == 1:
            return pd.Series(out[:, 0], index=index)
        columns = pd.RangeIndex(stop=out.shape[1], name='path')
        return pd.DataFrame(out, index=index, columns=columns)

    def update_symbol(self, symbol: tp.Label, **kwargs) -> tp.SeriesFrame:
        """更新GBM符号数据
        基于现有数据的最后价格继续生成新的GBM序列，即增量更新
        Args:
            symbol (tp.Label): 要更新的符号标识符
            **kwargs: 更新参数，会与原始下载参数合并
                常用参数包括：
                - end: 新的结束时间
                - mu: 新的漂移率（可以调整未来预期）
                - sigma: 新的波动率（可以调整波动程度）
                - 其他generate_symbol支持的参数

        download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
        download_kwargs['start'] = self.data[symbol].index[-1]
        _ = download_kwargs.pop('S0', None)
        S0 = self.data[symbol].iloc[-2] # 使用倒数第二个价格，确保价格连续性，避免价格跳跃
        _ = download_kwargs.pop('T', None)
        download_kwargs['seed'] = None # 清空随机种子，避免重复相同的随机模式
        kwargs = merge_dicts(download_kwargs, kwargs)
        return self.download_symbol(symbol, S0=S0, **kwargs)
```

## class YFData(Data)
**Yahoo Finance** 数据类。仅用于教育和研究目的，不建议用于实际交易决策。

1. 免费访问：无需 API 密钥，完全免费使用
2. 广泛覆盖：全球主要市场的股票、ETF、指数、商品
3. 多时间框架：从 1 分钟到月度的各种时间间隔
4. 实时性：提供 15-20 分钟延迟的准实时数据
5. 历史数据：可追溯数十年的历史价格数据

支持的资产类型：
- 股票：AAPL, GOOGL, MSFT 等美股和国际股票
- ETF：SPY, QQQ, ARKK 等交易所交易基金
- 指数：^GSPC (S&P 500), ^IXIC (NASDAQ)等
- 加密货币：BTC-USD, ETH-USD 等主流数字货币
- 外汇：EURUSD=X, GBPUSD=X 等货币对
- 商品：GC=F (黄金), CL=F (原油)等期货

数据格式：
返回标准的OHLCV数据，包含以下列：
- Open: 开盘价
- High: 最高价  
- Low: 最低价
- Close: 收盘价
- Volume: 成交量
- Dividends: 股息（如适用）
- Stock Splits: 股票拆分（如适用）

时区处理：
- 股票：通常使用市场本地时区（如美股为 EST/EDT）
- 加密货币：通常使用 UTC 时区
- 自动转换：vectorbt 会自动处理时区标准化

```python
class YFData(Data):

    @classmethod
    def download_symbol(cls,
                        symbol: tp.Label,
                        period: str = 'max',
                        start: tp.Optional[tp.DatetimeLike] = None,
                        end: tp.Optional[tp.DatetimeLike] = None,
                        ticker_kwargs: tp.KwargsLike = None,
                        **kwargs) -> tp.Frame:
        """
        下载单个符号的Yahoo Finance数据
        Args:
            symbol (str): 股票符号代码
            period (str): 预设时间周期，默认'max'
            start (any): 开始时间
            end (any): 结束时间  
            ticker_kwargs (dict): 传递给yfinance.Ticker的参数
            **kwargs: 传递给yfinance历史数据方法的参数
            
        Returns:
            tp.Frame: 包含OHLCV数据的DataFrame

        import yfinance as yf

        if start is not None:
            start = to_tzaware_datetime(start, tz=get_local_tz())
        if end is not None:
            end = to_tzaware_datetime(end, tz=get_local_tz())

        if ticker_kwargs is None:
            ticker_kwargs = {}
        return yf.Ticker(symbol, **ticker_kwargs).history(period=period, start=start, end=end, **kwargs)

    def update_symbol(self, symbol: tp.Label, **kwargs) -> tp.Frame:
        """
        更新指定符号的数据
        
        download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
        download_kwargs['start'] = self.data[symbol].index[-1]
        kwargs = merge_dicts(download_kwargs, kwargs)
        return self.download_symbol(symbol, **kwargs)
```

## class BinanceData(Data)
BinanceData 币安交易所数据类。基于 python-binance 库获取加密货币交易数据，是加密货币量化交易的首选数据源。

主要特点：
1. 高质量数据：直接来自交易所，数据准确可靠
2. 实时性强：提供近实时的市场数据
3. 多时间框架：支持1分钟到1月的各种K线间隔
4. 丰富信息：包含成交量、交易数量等详细市场数据
5. 免费使用：无需API密钥即可获取公开数据

支持的交易对：
- 主流币种：BTC/USDT, ETH/USDT, BNB/USDT等
- 山寨币：ADA/USDT, DOT/USDT, LINK/USDT等  
- 稳定币对：BTC/BUSD, ETH/BUSD等
- 法币对：BTC/EUR, ETH/GBP等

数据格式：
返回详细的K线数据，包含以下字段：
- Open: 开盘价
- High: 最高价
- Low: 最低价  
- Close: 收盘价
- Volume: 基础资产成交量
- Close time: 收盘时间
- Quote volume: 计价资产成交量
- Number of trades: 交易笔数
- Taker base volume: 主动买入基础资产量
- Taker quote volume: 主动买入计价资产量

### download
覆盖 `Data` 的 `download`： 使用 `download_symbol` 下载参数 `symbols` 对应的数据，然后使用 `from_data` 构建一个新的 `BinanceData` 类型的实例并返回。

Args:
- symbols: 要下载的符号列表
- client: 可选的Binance客户端实例
- **kwargs: 其他下载参数

Returns:
- BinanceDataT: 包含下载数据的BinanceData实例
```python
@classmethod
def download(cls: tp.Type[BinanceDataT],
                symbols: tp.Labels,
                client: tp.Optional["ClientT"] = None,
                **kwargs) -> BinanceDataT:
    from binance.client import Client
    from vectorbt._settings import settings
    binance_cfg = settings['data']['binance']

    client_kwargs = dict()
    # 获取 kwargs 中指定的 Client 构造函数的参数
    for k in get_func_kwargs(Client):
        if k in kwargs:
            client_kwargs[k] = kwargs.pop(k)
    client_kwargs = merge_dicts(binance_cfg, client_kwargs)
    if client is None:
        client = Client(**client_kwargs)
    return super(BinanceData, cls).download(symbols, client=client, **kwargs)
```

### download_symbol
实现 `Data` 的 `download_symbol`：下载 `symbol` 对应符号的新数据。

```python
@classmethod
def download_symbol(cls,
                    symbol: str,
                    client: tp.Optional["ClientT"] = None,
                    interval: str = '1d',
                    start: tp.DatetimeLike = 0,
                    end: tp.DatetimeLike = 'now UTC',
                    delay: tp.Optional[float] = 500,
                    limit: int = 500,
                    show_progress: bool = True,
                    tqdm_kwargs: tp.KwargsLike = None) -> tp.Frame:
    if client is None:
        raise ValueError("client must be provided")

    if tqdm_kwargs is None:
        tqdm_kwargs = {}
    start_ts = datetime_to_ms(to_tzaware_datetime(start, tz=get_utc_tz()))
    try:
        first_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=1,
            startTime=0,
            endTime=None
        )
        first_valid_ts = first_data[0][0]
        next_start_ts = start_ts = max(start_ts, first_valid_ts)
    except:
        next_start_ts = start_ts
    end_ts = datetime_to_ms(to_tzaware_datetime(end, tz=get_utc_tz()))

    def _ts_to_str(ts: tp.DatetimeLike) -> str:
        return str(pd.Timestamp(to_tzaware_datetime(ts, tz=get_utc_tz())))

    data: tp.List[list] = []
    with tqdm(disable=not show_progress, **tqdm_kwargs) as pbar:
        pbar.set_description(_ts_to_str(start_ts))
        while True:
            # Fetch the klines for the next interval
            next_data = client.get_klines(
                symbol=symbol,
                interval=interval,
                limit=limit,
                startTime=next_start_ts,
                endTime=end_ts
            )
            if len(data) > 0:
                next_data = list(filter(lambda d: next_start_ts < d[0] < end_ts, next_data))
            else:
                next_data = list(filter(lambda d: d[0] < end_ts, next_data))

            # Update the timestamps and the progress bar
            if not len(next_data):
                break
            data += next_data
            pbar.set_description("{} - {}".format(
                _ts_to_str(start_ts),
                _ts_to_str(next_data[-1][0])
            ))
            pbar.update(1)
            next_start_ts = next_data[-1][0]
            if delay is not None:
                time.sleep(delay / 1000)  # be kind to api

    df = pd.DataFrame(data, columns=[
        'Open time',
        'Open',
        'High',
        'Low',
        'Close',
        'Volume',
        'Close time',
        'Quote volume',
        'Number of trades',
        'Taker base volume',
        'Taker quote volume',
        'Ignore'
    ])
    df.index = pd.to_datetime(df['Open time'], unit='ms', utc=True)
    del df['Open time']
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df['Close time'] = pd.to_datetime(df['Close time'], unit='ms', utc=True)
    df['Quote volume'] = df['Quote volume'].astype(float)
    df['Number of trades'] = df['Number of trades'].astype(int)
    df['Taker base volume'] = df['Taker base volume'].astype(float)
    df['Taker quote volume'] = df['Taker quote volume'].astype(float)
    del df['Ignore']

    return df
```

### update_symbol
更新指定符号的 Binance 数据，实现数据的增量更新。
```python
def update_symbol(self, symbol: str, **kwargs) -> tp.Frame:
    download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
    download_kwargs['start'] = self.data[symbol].index[-1]
    download_kwargs['show_progress'] = False
    kwargs = merge_dicts(download_kwargs, kwargs)
    return self.download_symbol(symbol, **kwargs)
```

## class CCXTData(Data)
CCXT 多交易所数据类。基于ccxt库统一访问多个加密货币交易所。

主要优势：
1. 多交易所支持：统一接口访问 Binance、Coinbase、Kraken 等主流交易所
2. 标准化数据：自动处理不同交易所的数据格式差异
3. 简化接入：无需学习每个交易所的独特 API
4. 套利支持：便于跨交易所价格比较和套利策略
5. 容错处理：内建重试机制和错误处理

数据格式，返回标准OHLCV格式：
- Open: 开盘价
- High: 最高价
- Low: 最低价
- Close: 收盘价
- Volume: 成交量

### download_symbol

```python
@classmethod
def download_symbol(cls,
                    symbol: str,
                    exchange: tp.Union[str, "ExchangeT"] = 'binance',
                    config: tp.Optional[dict] = None,
                    timeframe: str = '1d',
                    start: tp.DatetimeLike = 0,
                    end: tp.DatetimeLike = 'now UTC',
                    delay: tp.Optional[float] = None,
                    limit: tp.Optional[int] = 500,
                    retries: int = 3,
                    show_progress: bool = True,
                    params: tp.Optional[dict] = None,
                    tqdm_kwargs: tp.KwargsLike = None) -> tp.Frame:
    import ccxt
    from vectorbt._settings import settings
    ccxt_cfg = settings['data']['ccxt']

    if config is None:
        config = {}
    if tqdm_kwargs is None:
        tqdm_kwargs = {}
    if params is None:
        params = {}
    if isinstance(exchange, str):
        if not hasattr(ccxt, exchange):
            raise ValueError(f"Exchange {exchange} not found")
        # Resolve config
        default_config = {}
        for k, v in ccxt_cfg.items():
            # Get general (not per exchange) settings
            if k in ccxt.exchanges:
                continue
            default_config[k] = v
        if exchange in ccxt_cfg:
            default_config = merge_dicts(default_config, ccxt_cfg[exchange])
        config = merge_dicts(default_config, config)
        exchange = getattr(ccxt, exchange)(config)
    else:
        if len(config) > 0:
            raise ValueError("Cannot apply config after instantiation of the exchange")
    if not exchange.has['fetchOHLCV']:
        raise ValueError(f"Exchange {exchange} does not support OHLCV")
    if timeframe not in exchange.timeframes:
        raise ValueError(f"Exchange {exchange} does not support {timeframe} timeframe")
    if exchange.has['fetchOHLCV'] == 'emulated':
        warnings.warn("Using emulated OHLCV candles", stacklevel=2)

    def _retry(method):
        @wraps(method)
        def retry_method(*args, **kwargs):
            for i in range(retries):
                try:
                    return method(*args, **kwargs)
                except (ccxt.NetworkError, ccxt.ExchangeError) as e:
                    if i == retries - 1:
                        raise e
                if delay is not None:
                    time.sleep(delay / 1000)

        return retry_method

    @_retry
    def _fetch(_since, _limit):
        return exchange.fetch_ohlcv(
            symbol,
            timeframe=timeframe,
            since=_since,
            limit=_limit,
            params=params
        )

    start_ts = datetime_to_ms(to_tzaware_datetime(start, tz=get_utc_tz()))
    try:
        first_data = _fetch(0, 1)
        first_valid_ts = first_data[0][0]
        next_start_ts = start_ts = max(start_ts, first_valid_ts)
    except:
        next_start_ts = start_ts
    end_ts = datetime_to_ms(to_tzaware_datetime(end, tz=get_utc_tz()))

    def _ts_to_str(ts):
        return str(pd.Timestamp(to_tzaware_datetime(ts, tz=get_utc_tz())))

    data: tp.List[list] = []
    with tqdm(disable=not show_progress, **tqdm_kwargs) as pbar:
        pbar.set_description(_ts_to_str(start_ts))
        while True:
            # Fetch the klines for the next interval
            next_data = _fetch(next_start_ts, limit)
            if len(data) > 0:
                next_data = list(filter(lambda d: next_start_ts < d[0] < end_ts, next_data))
            else:
                next_data = list(filter(lambda d: d[0] < end_ts, next_data))

            if not len(next_data):
                break
            data += next_data
            pbar.set_description("{} - {}".format(
                _ts_to_str(start_ts),
                _ts_to_str(next_data[-1][0])
            ))
            pbar.update(1)
            next_start_ts = next_data[-1][0]
            if delay is not None:
                time.sleep(delay / 1000)  # be kind to api

    df = pd.DataFrame(data, columns=[
        'Open time',
        'Open',
        'High',
        'Low',
        'Close',
        'Volume'
    ])
    df.index = pd.to_datetime(df['Open time'], unit='ms', utc=True)
    del df['Open time']
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)

    return df
```

### update_symbol

```python
def update_symbol(self, symbol: str, **kwargs) -> tp.Frame:
    download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
    download_kwargs['start'] = self.data[symbol].index[-1]
    download_kwargs['show_progress'] = False
    kwargs = merge_dicts(download_kwargs, kwargs)
    return self.download_symbol(symbol, **kwargs)
```

## class AlpacaData(Data)
Alpaca 证券数据类。基于alpaca-py 库获取美股和加密货币数据。

主要特点：
1. 专业级数据：来自专业券商，数据质量高于免费源
2. 实时性强：提供近实时的市场数据（免费账户有15分钟延迟）
3. 多资产支持：美股、ETF、加密货币等
4. API友好：专为程序化交易设计的现代RESTful API
5. 免费使用：提供免费的纸质交易账户和数据访问

支持的资产类型：
- 美股：NYSE、NASDAQ上市的所有股票
- ETF：主要的交易所交易基金
- 加密货币：BTC、ETH等主流数字货币
- 期权：股票期权数据（高级功能）

```python
# 需要先注册Alpaca账户并获取API密钥
# 注册地址：https://app.alpaca.markets/signup

### download_symbol

```python
@classmethod
def download_symbol(cls,
                    symbol: str,
                    timeframe: str = '1d',
                    start: tp.DatetimeLike = 0,
                    end: tp.DatetimeLike = 'now UTC',
                    adjustment: tp.Optional[str] = 'all',
                    limit: int = 500,
                    feed: tp.Optional[str] = None,
                    **kwargs) -> tp.Frame:
    from vectorbt._settings import settings
    from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
    from alpaca.data.requests import CryptoBarsRequest, StockBarsRequest
    from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

    alpaca_cfg = settings['data']['alpaca']

    if "/" in symbol:
        REST = CryptoHistoricalDataClient
    else:
        REST = StockHistoricalDataClient

    client_kwargs = dict()
    for k in get_func_kwargs(REST):
        if k in kwargs:
            client_kwargs[k] = kwargs.pop(k)

    client_kwargs = merge_dicts(alpaca_cfg, client_kwargs)

    client = REST(**client_kwargs)

    _timeframe_units = {'d': TimeFrameUnit.Day, 'h': TimeFrameUnit.Hour, 'm': TimeFrameUnit.Minute}

    if len(timeframe) < 2:
        raise ValueError("invalid timeframe")

    amount_str = timeframe[:-1]
    unit_str = timeframe[-1]

    if not amount_str.isnumeric() or unit_str not in _timeframe_units:
        raise ValueError("invalid timeframe")

    amount = int(amount_str)
    unit = _timeframe_units[unit_str]

    _timeframe = TimeFrame(amount, unit)

    start_ts = to_tzaware_datetime(start, tz=get_utc_tz()).isoformat()
    end_ts = to_tzaware_datetime(end, tz=get_utc_tz()).isoformat()

    if "/" in symbol:
        df = client.get_crypto_bars(CryptoBarsRequest(
            symbol_or_symbols=symbol,
            timeframe=_timeframe,
            start=start_ts,
            end=end_ts,
            limit=limit,
        )).df
    else:
        df = client.get_stock_bars(StockBarsRequest(
            symbol_or_symbols=symbol,
            timeframe=_timeframe,
            start=start_ts,
            end=end_ts,
            adjustment=adjustment,
            limit=limit,
            feed=feed,
        )).df

    df.drop(['trade_count', 'vwap'], axis=1, errors='ignore', inplace=True)

    df.rename(columns={
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume',
    }, inplace=True)

    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)

    return df
```

### update_symbol

```python
def update_symbol(self, symbol: str, **kwargs) -> tp.Frame:
    download_kwargs = self.select_symbol_kwargs(symbol, self.download_kwargs)
    download_kwargs['start'] = self.data[symbol].index[-1]
    download_kwargs['show_progress'] = False
    kwargs = merge_dicts(download_kwargs, kwargs)
    return self.download_symbol(symbol, **kwargs)
```

```python
